# 状态树

## 1 水桶与水

- 两个水桶，容积分别为7升和3升，有无限多的水，问如何能获得2升的水? 

思路：

两个水桶，初始状态都为0升水。

In [4]:
M = (7, 3)
b = [0 , 0]

定义各种加水、倒水的函数：把桶填满，把桶中水全部倒出，把水从一个桶倒入另一个桶

In [5]:
# 把桶填满
def Fill(idx):
    b[idx] = M[idx]
    return

# 把桶倒空
def Empty(idx):
    b[idx] = 0
    return

# 从fidx桶倒入tidx桶，分两种情况：能填满与不能填满
def transfer(fidx, tidx):
    if(b[tidx]!=M[tidx] and b[fidx]>0):
        if (b[fidx]+b[tidx])>=M[tidx]:
            b[fidx] = b[fidx]+b[tidx]-M[tidx]
            b[tidx] = M[tidx]
        else:
            b[tidx] = b[fidx]+b[tidx]
            b[fidx] = 0
    return 

声明状态字典，用来存储状态值，如果出现过这种状态，状态字典里就有这个状态的key，相应的value值表示经过几个步骤可以到达该状态；否则字典里没有这个key，需要添加进状态字典里。

In [11]:
stat = {}

每次加水或倒水都会生成新的状态，如果这个生成的状态已经存在过了，那么就不需要更新该状态了，否则需要把状态放入状态表中。

In [16]:
def insertStat(b0, b1, stepnum):
    s = (b0, b1)
    if (not s in stat):
        stat[s] = stepnum

状态集合初始为第0级，就是什么都不做的状态，两个桶都是0，层级数也设为0。

再进一步倒水或加水都会形成新的一级。

In [17]:
insertStat(0,0,0)
print(stat)

{(0, 0): 0}


In [ ]:
sidx = 0
for _, value in stat.items():
    if value == sidx:
        
Fill(0)
print(stat)

- 有3个水桶的容量为8升，5升和3升。 8升水桶里面装满了水。如果利用另外两个桶，得到4升水？ (没有无限水)

其他相似问题
- 卖牛奶的人有两桶10升的牛奶，两个顾客各带容器去买2升牛奶，一个带的是5升的容器，一个带的是4升的容器，这位卖牛奶的人应该如何解决问题？
- 三个没有刻度，容积分别为160升，119升和77升的空桶，和无限多的水，想要量出76升的水，至少需要进行多少次操作？

## 2 狼羊菜过河

- 一个农夫带着狼，羊和白菜想要过河，河边有一艘小船，只有农夫可以驾驶小船，而且农夫每次只能带一样东西过河，已知如果农夫不在旁边，那么，狼会吃掉羊，羊会吃掉白菜，农夫应该怎么过河？

- 一个猎人要带一棵胡萝卜，一只狗，两只老虎和一只羊过河，现在小船上只有三个座位，猎人最多只能带两样东西过去。没有人或狗在，老虎会吃羊，狗和羊单独在一起就会吵架，羊和胡萝卜在一起就会吃掉胡萝卜，有什么办法可以一起过河？


## 3 华容道

## 4 博弈树

## 5 简单AI